In [1]:
from classes import *

In [2]:
def test(a, b, c):
    qbm = QBM(a, b, c)
    return qbm

In [ ]:
a = test(50, 410, "LeapHybridSampler")
a.read_cmap()
a.read_images()
a.make_steps(1)

Step 2 began!
Got respose  10  out of 400
